In [1]:
import psycopg2
import numpy as np
import pandas as pd

In [3]:
import plotly.express as px
import plotly.graph_objs as go

In [5]:
from datetime import datetime, time

In [7]:
host="192.168.2.23"
port=5432
user="amt"
dbname="qdap_test"

In [9]:
def make_connection_to_db(host, port, user, dbname):
    conn = psycopg2.connect(host= host, port=port, user=user, dbname=dbname)
    cursor = conn.cursor()
    return cursor, conn

In [25]:
def fetch_options_data_timeframe(cursor, symbol, expiry_type, start_date, end_date):
    cursor.execute(
        f'''
            SELECT *
            FROM ohlcv_options_per_minute oopm
            WHERE symbol = '{symbol}' 
            AND oopm.expiry_type = '{expiry_type}'
            AND date_timestamp >= '{start_date}'
            AND date_timestamp <= '{end_date}'
            ORDER BY date_timestamp ASC;
        '''
    )
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

In [27]:
def fetch_futures_data_timeframe(cursor, symbol, expiry_type, start_date, end_date):
    query = f'''
        SELECT *
        FROM ohlcv_future_per_minute ofpm
        WHERE ofpm.symbol = '{symbol}'
        AND ofpm.expiry_type = '{expiry_type}'
        AND date_timestamp >= '{start_date}'
        AND date_timestamp <= '{end_date}'
        ORDER BY date_timestamp ASC;
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

In [43]:
def fetch(host, port, user, dbname, symbols, expiry_type_futures, expiry_type_options, start_date, end_date):
    cursor, conn = make_connection_to_db(host, port, user, dbname)
    dictionary_futures = {}
    dictionary_options = {}
    for symbol in symbols:
        df_futures = fetch_futures_data_timeframe(cursor, symbol, expiry_type_futures, start_date, end_date)
        df_futures['date_timestamp'] = pd.to_datetime(df_futures['date_timestamp'])
        df_futures['expiry'] = pd.to_datetime(df_futures['expiry'])
        
        df_options = fetch_options_data_timeframe(cursor, symbol, expiry_type_options, start_date, end_date)
        df_options['date_timestamp'] = pd.to_datetime(df_options['date_timestamp'])
        df_options['expiry'] = pd.to_datetime(df_options['expiry'])
        
        df_options = df_options[df_options['date_timestamp'].dt.date == df_options['expiry'].dt.date]
        expiries = pd.to_datetime(df_options['expiry']).dt.date
        df_futures = df_futures[df_futures['date_timestamp'].dt.date.isin(expiries)]
        
        dictionary_futures[symbol] = df_futures
        dictionary_options[symbol] = df_options
    cursor.close()
    conn.close()
    return dictionary_futures, dictionary_options

In [49]:
symbols = ["BANKNIFTY", "NIFTY", "FINNIFTY", "MIDCAPNIFTY"]
moneyness_strike = 0
fund_locked = 1000 # inr
fund_locked *= 100
transaction_cost = 11.5
slippage = 10
start_date = '2021-01-01'
end_date = '2022-01-01'
expiry_type_options = 'IW1'
expiry_type_futures = 'I'

In [51]:
DICT_FUTURES, DICT_OPTIONS = fetch(host, port, user, dbname, symbols, expiry_type_futures, expiry_type_options, start_date, end_date)

In [52]:
dict_futures = DICT_FUTURES.copy()
dict_options = DICT_OPTIONS.copy()

In [79]:
def fill_df_make_continuous_excluding_market_holidays(df, market_holidays_csv_path = r'C:\Users\user4\Desktop\exchange_holidays.csv'):
    market_holidays_df = pd.read_csv(market_holidays_csv_path, parse_dates=['holiday_date'])
    market_holidays = market_holidays_df['holiday_date'].dt.date.tolist()
    start_date = df['date_timestamp'].iloc[0]
    end_date = df['date_timestamp'].iloc[-1]
    all_days = pd.date_range(start=start_date, end=end_date, freq='B')
    trading_holidays = all_days.to_series().apply(lambda x: x.date() in market_holidays)
    trading_days = all_days[~trading_holidays]
    
    # Generate a complete range of trading minutes for each trading day
    trading_minutes = pd.date_range(start='09:15:00', end='15:29:00', freq='min').time
    
    # Create a complete index of trading timestamps
    complete_index = pd.DatetimeIndex([pd.Timestamp.combine(day, time) for day in trading_days for time in trading_minutes])
    df = df.reindex(complete_index).ffill()
    
    return df

In [89]:
def parse_options(df_options):
    df_calls = df_options[(df_options['opt_type'] == 'CE')]
    df_calls.set_index('date_timestamp', inplace=True)
    df_calls_open = df_calls.pivot(columns='strike', values='open').ffill()
    df_calls_close = df_calls.pivot(columns='strike', values='close').ffill()
    call_strikes = np.array(df_calls_close.columns, dtype=int)
    
    df_puts  = df_options[(df_options['opt_type'] == 'PE')]
    df_puts.set_index('date_timestamp', inplace=True)
    df_puts_open = df_puts.pivot(columns='strike', values='open').ffill()
    df_puts_close = df_puts.pivot(columns='strike', values='close').ffill()
    put_strikes = np.array(df_puts_close.columns, dtype=int)
    return [df_puts_open, df_puts_close], [df_calls_open, df_calls_close], [put_strikes, call_strikes]

In [87]:
for symbol in symbols:
    df_futures_symbol = dict_futures[symbol]
    df_options_symbol = dict_options[symbol]
    filled_df_futures = fill_df_make_continuous_excluding_market_holidays(df_futures_symbol)
    filled_df_options = fill_df_make_continuous_excluding_market_holidays(df_options_symbol)
    parsed_puts, parsed_calls, strikes = parse_options(filled_df_options)
    synthetic_futures = []
    for timestamp in filled_df_futures['date_timestamp']:
        futures_price = filled_df_futures['close'].loc[timestamp]
        k_p = np.argmin(abs(strikes[0]-futures_price))
        k_c = np.argmin(abs(strikes[1]-futures_price))
        synthetic_close: parsed_calls[k_c].loc[timestamp] - parsed_puts[k_p].loc[timestamp] + (k_p + k_c)/2
        synthetic_futures.append({
            "synthetic_close" : synthetic_close,
            "futures_close": futures_price,
            "call_strike": k_c,
            "put_strike": k_p
        })

C:\Users\user4\AppData\Local\Temp\5\ipykernel_14464\3253392119.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.reindex(complete_index).ffill()
C:\Users\user4\AppData\Local\Temp\5\ipykernel_14464\3253392119.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.reindex(complete_index).ffill()


KeyError: NaT